In [3]:
import pandas as pd
import numpy as np
import math

In [124]:
districts = pd.read_csv('./moscowpolicedata/districts.csv', delimiter=',', thousands=' ', encoding='utf-8', na_values=[' '])

In [5]:
districts.head()

,upperdistrict,name,population2015,population2014,population2013,population2012,population2011,crimebook2015,crimebook2014,crimebook2013,...,publicplaces2014,publicplaces2013,publicplaces2012,publicplaces2011,street2015,street2014,street2013,street2012,street2011,sources
0,ЦАО,Арбат,34 012,31 286,30 076,29 108,28 536,NaN,NaN,12735.0,...,453.0,336.0,580.0,NaN,NaN,226.0,198.0,326.0,NaN,https://github.com/openpolicerussia/moscowpoli...
1,ЦАО,Басманный,110 042,109 648,109 514,109 215,109 086,48201.0,36663.0,34510.0,...,667.0,606.0,710.0,NaN,562.0,366.0,391.0,458.0,NaN,https://github.com/openpolicerussia/moscowpoli...
2,ЦАО,Замоскворечье,57 722,57 012,56 937,56 483,56 098,31785.0,25935.0,22425.0,...,792.0,445.0,679.0,NaN,622.0,508.0,315.0,442.0,NaN,https://github.com/openpolicerussia/moscowpoli...
3,ЦАО,Красносельский,48 324,48 104,48 086,47 964,47 839,31295.0,25914.0,21796.0,...,702.0,547.0,NaN,NaN,709.0,404.0,317.0,NaN,NaN,https://github.com/openpolicerussia/moscowpoli...
4,ЦАО,Мещанский,60 014,59 337,59 276,58 936,58 514,NaN,28000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://github.com/openpolicerussia/moscowpoli...


In [7]:
import pandas_profiling

In [8]:
pandas_profiling.ProfileReport(districts)

Number of variables,88
Number of observations,135
Total Missing (%),18.3%
Total size in memory,92.9 KiB
Average record size in memory,704.6 B
Numeric,50
Categorical,7
Date,0
Text (Unique),2
Rejected,29
Distinct count,40


In [12]:
districts[['hijacking2011', 'economiccrimes2011']].dropna()

,hijacking2011,economiccrimes2011
69,57.0,122.0
87,14.0,24.0


Выводы:
* наполненность данными одних и тех же показателей по годам сильно различается
* сильная корреляция возникает из-за того, что значений крайне мало

Нормализуем данные: выделяем год и показатель в отдельные колонки

In [16]:
districts.columns

Index(['upperdistrict', 'name', 'population2015', 'population2014',
       'population2013', 'population2012', 'population2011', 'crimebook2015',
       'crimebook2014', 'crimebook2013', 'crimebook2012', 'crimebook2011',
       'crimes2015', 'crimes2014', 'crimes2013', 'crimes2012', 'crimes2011',
       'economiccrimes2015', 'economiccrimes2014', 'economiccrimes2013',
       'economiccrimes2012', 'economiccrimes2011', 'gravecrimes2015',
       'gravecrimes2014', 'gravecrimes2013', 'gravecrimes2012',
       'gravecrimes2011', 'murders2015', 'murders2014', 'murders2013',
       'murders2012', 'murders2011', 'healthdamage2015', 'healthdamage2014',
       'healthdamage2013', 'healthdamage2012', 'healthdamage2011',
       'robbing2015', 'robbing2014', 'robbing2013', 'robbing2012',
       'robbing2011', 'banditry2015', 'banditry2014', 'banditry2013',
       'banditry2012', 'banditry2011', 'thefts2015', 'thefts2014',
       'thefts2013', 'thefts2012', 'thefts2011', 'burglary2015',
       'bur

In [53]:
import re

indicators_with_years = list(filter(lambda s: re.match('\w+\d{4}', s), districts.columns))
other = list(filter(lambda s: not re.match('\w+\d{4}', s), districts.columns))
indicator_pairs = []
for s in indicators_with_years:
    for it in re.finditer('(\w+)(\d{4})', s):
        indicator_pairs.append(tuple((it.group(1), it.group(2))))

In [42]:
def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

In [105]:
other

['upperdistrict', 'name', 'sources']

In [126]:
data = pd.DataFrame(columns = ['upperdistrict', 'name', 'year', 'indicator', 'value'])

for idx, row in districts.iterrows():
    for (i, y) in indicator_pairs:
        d = {}
        d['upperdistrict'] = [row['upperdistrict']]
        d['name'] = [row['name']]
        d['indicator'] =[i]
        d['year'] = [y]
#         d['value'] = [math.nan if is_nan(row[i+y]) else int(str(row[i+y]).replace(' ', ''))]
        d['value'] = [row[i+y]]
        new = pd.DataFrame.from_dict(data=d, orient='columns')
        data = pd.concat([data, new])

data.head()

,indicator,name,upperdistrict,value,year
0,population,Арбат,ЦАО,34012.0,2015
0,population,Арбат,ЦАО,31286.0,2014
0,population,Арбат,ЦАО,30076.0,2013
0,population,Арбат,ЦАО,29108.0,2012
0,population,Арбат,ЦАО,28536.0,2011


In [128]:
pandas_profiling.ProfileReport(data)

Number of variables,6
Number of observations,11475
Total Missing (%),5.0%
Total size in memory,538.0 KiB
Average record size in memory,48.0 B
Numeric,1
Categorical,4
Date,0
Text (Unique),0
Rejected,1
Constant value,0


In [119]:
data.shape

(1374, 5)

In [133]:
data.to_csv('work/data/districts_normalized.csv', sep=',', encoding='utf-8')

In [29]:
from tqdm import tqdm, trange

In [48]:
data = pd.read_csv('../data/districts_normalized.csv', sep=',', encoding='utf-8') \
    .drop(['Unnamed: 0'], axis=1) \
    .dropna()
    
data[['year', 'value']] = data[['year', 'value']].astype(int)

In [49]:
data.head()

,indicator,name,upperdistrict,value,year
0,population,Арбат,ЦАО,34012,2015
1,population,Арбат,ЦАО,31286,2014
2,population,Арбат,ЦАО,30076,2013
3,population,Арбат,ЦАО,29108,2012
4,population,Арбат,ЦАО,28536,2011


In [37]:
import requests
import json

In [51]:
index_name = 'crimestat'
record_type = 'stat_record'

requests.delete('http://localhost:9200/{}'.format(index_name))
for i in trange(data.shape[0]):
    row = data.iloc[i]
    if is_nan(row['value']):
        continue
    d={'year': int(row['year']), 'indicator': row['indicator'], 
                        'name': row['name'], 'upperdistrict': row['upperdistrict'], 
                  'value': int(row['value'])}
    req = requests.post('http://localhost:9200/{}/{}'.format(index_name, record_type), data=json.dumps(d))

100%|██████████| 8030/8030 [03:20<00:00, 40.00it/s]
